### MNIST with Estimators

In [10]:
import numpy as np
import tensorflow as tf
import argparse
import sys
import math
import gzip
import os
import shutil
from six.moves import urllib
from tensorflow.python.platform import gfile

In [4]:
#Enable Logging
tf.logging.set_verbosity(tf.logging.INFO)
# tf.logging.log(logging.INFO, "Tensorflow Version"+tf.__version__)

In [5]:
def maybe_download_and_extract(filename, work_directory, source_url):
    """
    To download &/ extract the zip files from a given working directory
    Args:
        filename: File to be extracted
        work_directory: Path to the directory
        source_url: URL to download the zip files
    Returns:
        unzipped_filepath: Path to the unzipped files
    """    
    if filename[-3:] == ".gz":
        unzipped_filename = filename[:-3]
    else:
        unzipped_filename = filename

    if not gfile.Exists(work_directory):
        gfile.MakeDirs(work_directory)

    filepath = os.path.join(work_directory, filename)
    unzipped_filepath = os.path.join(work_directory, unzipped_filename)

    if not gfile.Exists(unzipped_filepath):
        urllib.request.urlretrieve(source_url, filepath)

        if not filename == unzipped_filename:
            with gzip.open(filepath, 'rb') as f_in:
                with open(unzipped_filepath, 'wb') as f_out: # remove .gz
                    shutil.copyfileobj(f_in, f_out)

        with gfile.GFile(filepath) as f:
            size = f.size()
        print('Successfully downloaded and unzipped', filename, size, 'bytes.')
    return unzipped_filepath

In [6]:
def read_labels(tf_bytestring):
    """
    Decodes the labels in string format to Tensor
    Args:
        tf_string: Labels in string format
    Returns:
        labels: As tf.float32
    """
    label = tf.decode_raw(tf_bytestring, tf.uint8)
    return (tf.reshape(label, []))

In [7]:
def read_images(tf_bytestring):
    """
    Decodes the input data in string format to Tensor
    Args:
        tf_string: Images in string Format
    Returns:
        image: as tf.float32
    """
    image = tf.decode_raw(tf_bytestring, tf.uint8)
    return(tf.cast(image, tf.float32))

In [8]:
def load_data_mnist(data_dir):
    """
    May download and extract the data files
    Args:
        data_dir: Path to where dataset files are stored
    Returns:
        train_data:
        train_labels:
        test_data:
        test_labels:
    """
    train_images_file = 'train-images-idx3-ubyte.gz'
    local_train_images_file = maybe_download_and_extract(train_images_file, data_dir, SOURCE_URL+train_images_file)
    train_labels_file = 'train-labels-idx1-ubyte.gz'
    local_train_labels_file = maybe_download_and_extract(train_labels_file, data_dir, SOURCE_URL+train_labels_file)
    test_images_file = 't10k-images-idx3-ubyte.gz'
    local_test_images_file = maybe_download_and_extract(test_images_file, data_dir, SOURCE_URL+test_images_file)
    test_labels_file = 't10k-labels-idx1-ubyte.gz'
    local_test_labels_file = maybe_download_and_extract(test_labels_file, data_dir, SOURCE_URL+test_labels_file)
    return local_train_images_file, local_test_images_file, local_train_labels_file, local_test_labels_file

In [9]:
def load_data(imagefile, labelfile):
    """
    Creates interleaved images and labels dataset
    Args:
        imagefile: To create tensor of image_data
        labelfile: To create tensor of label_data
    Returns:
        dataset: the combined dataset of images and labels
    """
    image_data = tf.FixedLengthRecordDataset(imagefile, 28*28, header_bytes=16, buffer_size=1024*16).map(read_images)
    label_data = tf.FixedLengthRecordDataset(labelfile, 8, header_bytes=16, buffer_size=1024*16).map(read_labels)
    dataset = tf.data.Dataset.zip((image_data, label_data))
    return dataset

In [ ]:
def main(args):
    """
    """
    parser = argparse.ArgumentParser()